In [1]:
import os
os.getpid()

17955

In [12]:
import sys
sys.path.append('..')
# import bioroboost as br
from importlib import reload
import re
import tools as brt
import Bio.SeqIO
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Alphabet import generic_dna, generic_protein, IUPAC, ProteinAlphabet, DNAAlphabet
import pandas as pd

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# this is the WT sequence of the luciferase
seqBio = Seq('ATGGAAGATGCCAAAAACATTAAGAAGGGCCCAGCGCCATTCTACCCACTCGAAGACGGGACCGCCGGCGAGCAGCTGCACAAAGCCATGAAGCGCTACGCCCTGGTGCCCGGCACCATCGCCTTTACCGACGCACATATCGAGGTGGACATTACCTACGCCGAGTACTTCGAGATGAGCGTTCGGCTGGCAGAAGCTATGAAGCGCTATGGGCTGAATACAAACCATCGGATCGTGGTGTGCAGCGAGAATAGCTTGCAGTTCTTCATGCCCGTGTTGGGTGCCCTGTTCATCGGTGTGGCTGTGGCCCCAGCTAACGACATCTACAACGAGCGCGAGCTGCTGAACAGCATGGGCATCAGCCAGCCCACCGTCGTATTCGTGAGCAAGAAAGGGCTGCAAAAGATCCTCAACGTGCAAAAGAAGCTACCGATCATACAAAAGATCATCATCATGGATAGCAAGACCGACTACCAGGGCTTCCAAAGCATGTACACCTTCGTGACTTCCCATTTGCCACCCGGCTTCAACGAGTACGACTTCGTGCCCGAGAGCTTCGACCGGGACAAAACCATCGCCCTGATCATGAACAGTAGTGGCAGTACCGGATTGCCCAAGGGCGTAGCCCTACCGCACCGCACCGCTTGTGTCCGATTCAGTCATGCCCGCGACCCCATCTTCGGCAACCAGATCATCCCCGACACCGCTATCCTCAGCGTGGTGCCATTTCACCACGGCTTCGGCATGTTCACCACGCTGGGCTACTTGATCTGCGGCTTTCGGGTCGTGCTCATGTACCGCTTCGAGGAGGAGCTATTCTTGCGCAGCTTGCAAGACTATAAGATTCAATCTGCCCTGCTGGTGCCCACACTATTTAGCTTCTTCGCTAAGAGCACTCTCATCGACAAGTACGACCTAAGCAACTTGCACGAGATCGCCAGCGGCGGGGCGCCGCTCAGCAAGGAGGTAGGTGAGGCCGTGGCCAAACGCTTCCACCTACCAGGCATCCGCCAGGGCTACGGCCTGACAGAAACAACCAGCGCCATTCTGATCACCCCCGAAGGGGACGACAAGCCTGGCGCAGTAGGCAAGGTGGTGCCCTTCTTCGAGGCTAAGGTGGTGGACTTGGACACCGGTAAGACACTGGGTGTGAACCAGCGCGGCGAGCTGTGCGTCCGTGGCCCCATGATCATGAGCGGCTACGTTAACAACCCCGAGGCTACAAACGCTCTCATCGACAAGGACGGCTGGCTGCACAGCGGCGACATCGCCTACTGGGACGAGGACGAGCACTTCTTCATCGTGGACCGGCTGAAGAGCCTGATCAAATACAAGGGCTACCAGGTAGCCCCAGCCGAACTGGAGAGCATCCTGCTGCAACACCCCAACATCTTCGACGCCGGGGTCGCCGGCCTGCCCGACGACGATGCCGGCGAGCTGCCCGCCGCAGTCGTCGTGCTGGAACACGGTAAAACCATGACCGAGAAGGAGATCGTGGACTATGTGGCCAGCCAGGTTACAACCGCCAAGAAGCTGCGCGGTGGTGTTGTGTTCGTGGACGAGGTGCCTAAAGGACTGACCGGCAAGTTGGACGCCCGCAAGATCCGCGAGATTCTCATTAAGGCCAAGAAGGGCGGCAAGATCGCCGTGTAA', generic_dna)
seqBio

Seq('ATGGAAGATGCCAAAAACATTAAGAAGGGCCCAGCGCCATTCTACCCACTCGAA...TAA', DNAAlphabet())

# Codon usage

## Compute species codon usage table

implement codon score based on:

- CAI
- (for myco: codon translation speed based on ribosome density data?)

CAI of individual codons?

- create table of codon usage that will be used to compute the codon scores
- implement the checker in the backend tool to compute the codon scores along the sequence
    - note: we have to return a value for each single nucleotide position, so repeating the same score for each position in the triplet.

In [12]:
from mwTools.cai import split_6_codons_synonymous_families
from mwTools.cai import compute_CAI
from mwTools.cai import compute_codon_usage_series
from mwTools.cai import compute_codon_usage
from mwTools.cai import compute_CAI_df
from mwTools.bio import build_refCodonTable

### Mycoplasma

In [17]:
codonTableId = 4
codonTableBio = Bio.Data.CodonTable.unambiguous_dna_by_id[codonTableId]
print(codonTableBio)
refCodonTableDf = build_refCodonTable(codonTableBio)
refCodonTableDf.head()

Table 4 Mold Mitochondrial, Protozoan Mitochondrial, Coelenterate Mitochondrial, Mycoplasma, Spiroplasma, SGC3

  |  T      |  C      |  A      |  G      |
--+---------+---------+---------+---------+--
T | TTT F   | TCT S   | TAT Y   | TGT C   | T
T | TTC F   | TCC S   | TAC Y   | TGC C   | C
T | TTA L(s)| TCA S   | TAA Stop| TGA W   | A
T | TTG L(s)| TCG S   | TAG Stop| TGG W   | G
--+---------+---------+---------+---------+--
C | CTT L   | CCT P   | CAT H   | CGT R   | T
C | CTC L   | CCC P   | CAC H   | CGC R   | C
C | CTA L   | CCA P   | CAA Q   | CGA R   | A
C | CTG L(s)| CCG P   | CAG Q   | CGG R   | G
--+---------+---------+---------+---------+--
A | ATT I(s)| ACT T   | AAT N   | AGT S   | T
A | ATC I(s)| ACC T   | AAC N   | AGC S   | C
A | ATA I(s)| ACA T   | AAA K   | AGA R   | A
A | ATG M(s)| ACG T   | AAG K   | AGG R   | G
--+---------+---------+---------+---------+--
G | GTT V   | GCT A   | GAT D   | GGT G   | T
G | GTC V   | GCC A   | GAC D   | GGC G   | C
G | GTA V   | GC

,aa,i,j,alphabetical_sorted_index,aa_groups_sorted_index
codon,,,,,
CGT,R,5,7,52,0
CGC,R,6,7,53,1
CGA,R,7,7,54,2
CGG,R,8,7,55,3
AGA,R,11,7,58,4


In [18]:
aaCodonDict = refCodonTableDf.reset_index()[['codon', 'aa']].groupby('aa').agg(lambda x: list(x)).to_dict()['codon']
aaCodonDict

{'*': ['TAA', 'TAG'],
 'A': ['GCT', 'GCC', 'GCA', 'GCG'],
 'C': ['TGT', 'TGC'],
 'D': ['GAT', 'GAC'],
 'E': ['GAA', 'GAG'],
 'F': ['TTT', 'TTC'],
 'G': ['GGT', 'GGC', 'GGA', 'GGG'],
 'H': ['CAT', 'CAC'],
 'I': ['ATT', 'ATC', 'ATA'],
 'K': ['AAA', 'AAG'],
 'L': ['TTA', 'TTG', 'CTT', 'CTC', 'CTA', 'CTG'],
 'M': ['ATG'],
 'N': ['AAT', 'AAC'],
 'P': ['CCT', 'CCC', 'CCA', 'CCG'],
 'Q': ['CAA', 'CAG'],
 'R': ['CGT', 'CGC', 'CGA', 'CGG', 'AGA', 'AGG'],
 'S': ['TCT', 'TCC', 'TCA', 'TCG', 'AGT', 'AGC'],
 'T': ['ACT', 'ACC', 'ACA', 'ACG'],
 'V': ['GTT', 'GTC', 'GTA', 'GTG'],
 'W': ['TGA', 'TGG'],
 'Y': ['TAT', 'TAC']}

In [19]:
genomeFilePath = '/users/lserrano/mweber/Research_cloud/Mycoplasma_pneumoniae_experimental_data/Annotation/mpn_annotations.gbk'
genomeBio = next(Bio.SeqIO.parse(str(genomeFilePath), format="genbank"))
genomeBio

SeqRecord(seq=Seq('TATTTACCGACGAAATTAATACCATCAGGGTATTAAGATGCTACCAACGTGGTA...TAT', IUPACAmbiguousDNA()), id='NC_000912.1', name='NC_000912', description='Mycoplasma pneumoniae M129 chromosome, complete genome', dbxrefs=['BioProject:PRJNA57709'])

In [33]:
seqs = pd.Series([str(feat.extract(genomeBio).seq) for feat in genomeBio.features if feat.type == 'CDS'])
len(seqs)

713

In [36]:
codon_usage = compute_codon_usage(seqs, aaCodonDict, checkLengthMultipleOf3=True, verbose=0)
codon_usage.keys()

ERROR: seq length is not multiple of 3.


dict_keys(['codonFreq', 'synCodonCount', 'fij', 'wij', 'codonDatabase', 'codonCount'])

In [41]:
codon_wij = pd.Series({k:v for d in codon_usage['wij'].values() for k, v in d.items()}).sort_index()
codon_wij

AAA    1.000000
AAC    1.000000
AAG    0.841917
AAT    0.676809
ACA    0.476317
ACC    1.000000
ACG    0.362827
ACT    0.878552
AGA    0.375578
AGC    0.505336
AGG    0.265794
AGT    1.000000
ATA    0.120047
ATC    0.314391
ATG    1.000000
ATT    1.000000
CAA    1.000000
CAC    1.000000
CAG    0.412872
CAT    0.518207
CCA    1.000000
CCC    0.831872
CCG    0.605032
CCT    0.750667
CGA    0.233821
CGC    1.000000
CGG    0.467643
CGT    0.907935
CTA    0.270893
CTC    0.311886
         ...   
GAG    0.352339
GAT    1.000000
GCA    0.547403
GCC    0.653916
GCG    0.443364
GCT    1.000000
GGA    0.228410
GGC    0.423913
GGG    0.321769
GGT    1.000000
GTA    0.651530
GTC    0.518223
GTG    0.885792
GTT    1.000000
TAA    1.000000
TAC    1.000000
TAG    0.392233
TAT    0.801724
TCA    0.413834
TCC    0.453360
TCG    0.303360
TCT    0.391502
TGA    1.000000
TGC    0.392857
TGG    0.962225
TGT    1.000000
TTA    1.000000
TTC    0.292896
TTG    0.552034
TTT    1.000000
Length: 64, dtype: float

In [43]:
codon_wij.to_csv('../../mycoplasma_pneumoniae_M129_codon_wij.txt', sep='\t')

## Compute score of codon lists

In [44]:
seqBio

Seq('ATGGAAGATGCCAAAAACATTAAGAAGGGCCCAGCGCCATTCTACCCACTCGAA...TAA', DNAAlphabet())

In [45]:
codon_wij

AAA    1.000000
AAC    1.000000
AAG    0.841917
AAT    0.676809
ACA    0.476317
ACC    1.000000
ACG    0.362827
ACT    0.878552
AGA    0.375578
AGC    0.505336
AGG    0.265794
AGT    1.000000
ATA    0.120047
ATC    0.314391
ATG    1.000000
ATT    1.000000
CAA    1.000000
CAC    1.000000
CAG    0.412872
CAT    0.518207
CCA    1.000000
CCC    0.831872
CCG    0.605032
CCT    0.750667
CGA    0.233821
CGC    1.000000
CGG    0.467643
CGT    0.907935
CTA    0.270893
CTC    0.311886
         ...   
GAG    0.352339
GAT    1.000000
GCA    0.547403
GCC    0.653916
GCG    0.443364
GCT    1.000000
GGA    0.228410
GGC    0.423913
GGG    0.321769
GGT    1.000000
GTA    0.651530
GTC    0.518223
GTG    0.885792
GTT    1.000000
TAA    1.000000
TAC    1.000000
TAG    0.392233
TAT    0.801724
TCA    0.413834
TCC    0.453360
TCG    0.303360
TCT    0.391502
TGA    1.000000
TGC    0.392857
TGG    0.962225
TGT    1.000000
TTA    1.000000
TTC    0.292896
TTG    0.552034
TTT    1.000000
Length: 64, dtype: float

In [48]:
sequence = str(seqBio)
sequence

'ATGGAAGATGCCAAAAACATTAAGAAGGGCCCAGCGCCATTCTACCCACTCGAAGACGGGACCGCCGGCGAGCAGCTGCACAAAGCCATGAAGCGCTACGCCCTGGTGCCCGGCACCATCGCCTTTACCGACGCACATATCGAGGTGGACATTACCTACGCCGAGTACTTCGAGATGAGCGTTCGGCTGGCAGAAGCTATGAAGCGCTATGGGCTGAATACAAACCATCGGATCGTGGTGTGCAGCGAGAATAGCTTGCAGTTCTTCATGCCCGTGTTGGGTGCCCTGTTCATCGGTGTGGCTGTGGCCCCAGCTAACGACATCTACAACGAGCGCGAGCTGCTGAACAGCATGGGCATCAGCCAGCCCACCGTCGTATTCGTGAGCAAGAAAGGGCTGCAAAAGATCCTCAACGTGCAAAAGAAGCTACCGATCATACAAAAGATCATCATCATGGATAGCAAGACCGACTACCAGGGCTTCCAAAGCATGTACACCTTCGTGACTTCCCATTTGCCACCCGGCTTCAACGAGTACGACTTCGTGCCCGAGAGCTTCGACCGGGACAAAACCATCGCCCTGATCATGAACAGTAGTGGCAGTACCGGATTGCCCAAGGGCGTAGCCCTACCGCACCGCACCGCTTGTGTCCGATTCAGTCATGCCCGCGACCCCATCTTCGGCAACCAGATCATCCCCGACACCGCTATCCTCAGCGTGGTGCCATTTCACCACGGCTTCGGCATGTTCACCACGCTGGGCTACTTGATCTGCGGCTTTCGGGTCGTGCTCATGTACCGCTTCGAGGAGGAGCTATTCTTGCGCAGCTTGCAAGACTATAAGATTCAATCTGCCCTGCTGGTGCCCACACTATTTAGCTTCTTCGCTAAGAGCACTCTCATCGACAAGTACGACCTAAGCAACTTGCACGAGATCGCCAGCGGCGGGGCGCCGCTCAGCAAGGAGGTAGGTGAGGCCGTGGCCAAACGCTTCCACCTA

In [49]:
codon_usage_matrix_path = '../../mycoplasma_pneumoniae_M129_codon_wij.txt'

In [61]:
from mwTools.bio import extract_codons_list

In [63]:
codons = list(extract_codons_list(sequence, checkLengthMultipleOf3=False))
codons

['ATG',
 'GAA',
 'GAT',
 'GCC',
 'AAA',
 'AAC',
 'ATT',
 'AAG',
 'AAG',
 'GGC',
 'CCA',
 'GCG',
 'CCA',
 'TTC',
 'TAC',
 'CCA',
 'CTC',
 'GAA',
 'GAC',
 'GGG',
 'ACC',
 'GCC',
 'GGC',
 'GAG',
 'CAG',
 'CTG',
 'CAC',
 'AAA',
 'GCC',
 'ATG',
 'AAG',
 'CGC',
 'TAC',
 'GCC',
 'CTG',
 'GTG',
 'CCC',
 'GGC',
 'ACC',
 'ATC',
 'GCC',
 'TTT',
 'ACC',
 'GAC',
 'GCA',
 'CAT',
 'ATC',
 'GAG',
 'GTG',
 'GAC',
 'ATT',
 'ACC',
 'TAC',
 'GCC',
 'GAG',
 'TAC',
 'TTC',
 'GAG',
 'ATG',
 'AGC',
 'GTT',
 'CGG',
 'CTG',
 'GCA',
 'GAA',
 'GCT',
 'ATG',
 'AAG',
 'CGC',
 'TAT',
 'GGG',
 'CTG',
 'AAT',
 'ACA',
 'AAC',
 'CAT',
 'CGG',
 'ATC',
 'GTG',
 'GTG',
 'TGC',
 'AGC',
 'GAG',
 'AAT',
 'AGC',
 'TTG',
 'CAG',
 'TTC',
 'TTC',
 'ATG',
 'CCC',
 'GTG',
 'TTG',
 'GGT',
 'GCC',
 'CTG',
 'TTC',
 'ATC',
 'GGT',
 'GTG',
 'GCT',
 'GTG',
 'GCC',
 'CCA',
 'GCT',
 'AAC',
 'GAC',
 'ATC',
 'TAC',
 'AAC',
 'GAG',
 'CGC',
 'GAG',
 'CTG',
 'CTG',
 'AAC',
 'AGC',
 'ATG',
 'GGC',
 'ATC',
 'AGC',
 'CAG',
 'CCC',
 'ACC',
 'GTC',


In [66]:
matrix = codon_wij
matrix.at['AAA']

1.0

In [80]:
matrix
[matrix.at[c] for c in codons]

rs = pd.Series(codons).map(matrix).rolling(window=1, center=True).mean()

In [108]:
matrix = brt.load_matrix(codon_usage_matrix_path, residue_type='DNA', k=3)
matrix

,1
0,
AAA,1.000000
AAC,1.000000
AAG,0.841917
AAT,0.676809
ACA,0.476317
ACC,1.000000
ACG,0.362827
ACT,0.878552
AGA,0.375578


In [111]:
n = 3
rs = pd.Series(codons).map(matrix.iloc[:, 0].to_dict()).rolling(window=n, center=True).mean()
# convert codon position to nucleotide position, repeat codon score 
# for each nucleotide position in the codon
rs0 = rs.copy()
rs0.index = 3*rs0.index
rs1 = rs.copy()
rs1.index = 3*rs1.index + 1
rs2 = rs.copy()
rs2.index = 3*rs2.index + 2
rs = pd.concat([rs0, rs1, rs2]).sort_index()
# Convert to 1-based index
rs.index = rs.index + 1
rs = rs.to_dict()
rs

{1: nan,
 2: nan,
 3: nan,
 4: 1.0,
 5: 1.0,
 6: 1.0,
 7: 0.8846386369881835,
 8: 0.8846386369881835,
 9: 0.8846386369881835,
 10: 0.8846386369881835,
 11: 0.8846386369881835,
 12: 0.8846386369881835,
 13: 0.8846386369881835,
 14: 0.8846386369881835,
 15: 0.8846386369881835,
 16: 1.0,
 17: 1.0,
 18: 1.0,
 19: 0.947305674773486,
 20: 0.947305674773486,
 21: 0.947305674773486,
 22: 0.8946113495469721,
 23: 0.8946113495469721,
 24: 0.8946113495469721,
 25: 0.7025823640397256,
 26: 0.7025823640397256,
 27: 0.7025823640397256,
 28: 0.7552766892662396,
 29: 0.7552766892662396,
 30: 0.7552766892662396,
 31: 0.622425534965411,
 32: 0.622425534965411,
 33: 0.622425534965411,
 34: 0.8144545204726574,
 35: 0.8144545204726574,
 36: 0.8144545204726574,
 37: 0.5787532804457421,
 38: 0.5787532804457421,
 39: 0.5787532804457421,
 40: 0.7642987599730849,
 41: 0.7642987599730849,
 42: 0.7642987599730849,
 43: 0.7642987599730849,
 44: 0.7642987599730849,
 45: 0.7642987599730849,
 46: 0.7706286318013736,


In [116]:
brt.checker(sequence,
            elements='codon_adaptation', matrix_dict={'codon_adaptation':codon_usage_matrix_path},
            codon_table=4,
            circular=False, residue_type='DNA',
            indexed=True, outhandle='json', standardize=True,
           verbose=2)



--> Matrix scoring started for ../../mycoplasma_pneumoniae_M129_codon_wij.txt
	Matrix scoring: Loading scoring matrix...
matrix
             1
0            
AAA  1.000000
AAC  1.000000
AAG  0.841917
AAT  0.676809
ACA  0.476317
ACC  1.000000
ACG  0.362827
ACT  0.878552
AGA  0.375578
AGC  0.505336
AGG  0.265794
AGT  1.000000
ATA  0.120047
ATC  0.314391
ATG  1.000000
ATT  1.000000
CAA  1.000000
CAC  1.000000
CAG  0.412872
CAT  0.518207
CCA  1.000000
CCC  0.831872
CCG  0.605032
CCT  0.750667
CGA  0.233821
CGC  1.000000
CGG  0.467643
CGT  0.907935
CTA  0.270893
CTC  0.311886
..        ...
GAG  0.352339
GAT  1.000000
GCA  0.547403
GCC  0.653916
GCG  0.443364
GCT  1.000000
GGA  0.228410
GGC  0.423913
GGG  0.321769
GGT  1.000000
GTA  0.651530
GTC  0.518223
GTG  0.885792
GTT  1.000000
TAA  1.000000
TAC  1.000000
TAG  0.392233
TAT  0.801724
TCA  0.413834
TCC  0.453360
TCG  0.303360
TCT  0.391502
TGA  1.000000
TGC  0.392857
TGG  0.962225
TGT  1.000000
TTA  1.000000
TTC  0.292896
TTG  0.552034
T

{'codon_adaptation': '[{"start": 1, "score": NaN}, {"start": 2, "score": NaN}, {"start": 3, "score": NaN}, {"start": 4, "score": NaN}, {"start": 5, "score": NaN}, {"start": 6, "score": NaN}, {"start": 7, "score": 0.9134789777411376}, {"start": 8, "score": 0.9134789777411376}, {"start": 9, "score": 0.9134789777411376}, {"start": 10, "score": 0.9134789777411376}, {"start": 11, "score": 0.9134789777411376}, {"start": 12, "score": 0.9134789777411376}, {"start": 13, "score": 0.9134789777411376}, {"start": 14, "score": 0.9134789777411376}, {"start": 15, "score": 0.9134789777411376}, {"start": 16, "score": 0.9134789777411376}, {"start": 17, "score": 0.9134789777411376}, {"start": 18, "score": 0.9134789777411376}, {"start": 19, "score": 0.9604792560801145}, {"start": 20, "score": 0.9604792560801145}, {"start": 21, "score": 0.9604792560801145}, {"start": 22, "score": 0.9209585121602291}, {"start": 23, "score": 0.9209585121602291}, {"start": 24, "score": 0.9209585121602291}, {"start": 25, "score

# Shine-Dalgarno

## compute scores of example sequence

In [4]:
from ribosome_binding_site import compute_best_motif_hybridization_energy_in_seq
from ribosome_binding_site import compute_RBS_affinity
from ribosome_binding_site import RBS_CANONICAL

In [5]:
print(compute_best_motif_hybridization_energy_in_seq('AGGAGG', 'TATATCGGCAGGAGGTGCATGATCCTA', verbose=2))

motif_seq CCTCCT
subseq_list ['TATATC', 'ATATCG', 'TATCGG', 'ATCGGC', 'TCGGCA', 'CGGCAG', 'GGCAGG', 'GCAGGA', 'CAGGAG', 'AGGAGG', 'GGAGGT', 'GAGGTG', 'AGGTGC', 'GGTGCA', 'GTGCAT', 'TGCATG', 'GCATGA', 'CATGAT', 'ATGATC', 'TGATCC', 'GATCCT', 'ATCCTA']
energy_fold_list [['.............', 0.0], ['.............', 0.0], ['...((.....)).', -0.699999988079071], ['...((....))..', -1.100000023841858], ['...((...))...', -1.100000023841858], ['.((((..)).)).', -1.600000023841858], ['(((((.)).))).', -4.400000095367432], ['..((((..)))).', -3.0999999046325684], ['.(((((.))))).', -5.900000095367432], ['(((((()))))).', -8.600000381469727], ['(((((.)))))..', -8.800000190734863], ['((((..))))...', -5.099999904632568], ['.(.((())).)..', -1.600000023841858], ['.(.((.)).)...', -1.2000000476837158], ['.............', 0.0], ['.............', 0.0], ['.............', 0.0], ['.............', 0.0], ['.............', 0.0], ['.............', 0.0], ['.............', 0.0], ['.............', 0.0]]
(-8.800000190734863, 1

In [7]:
sequence = ('CCACTCGA'    # not transcribed
            'AGACGGGACCGCCGGCGAGCAGGTGCACAAAGCC'    # 5'UTR
            'ATGAAGCGCTACGCCCTGGTGCCCGGCACCATCGCCTTTACCGACGCACATATCGAGGTGGACATTACCTACGCCGAGTACTTCTAA')    # CDS
print(sequence[:8])
print(sequence[42:45])
print(sequence[26:26+6])

CCACTCGA
ATG
GCAGGT


In [8]:
print(sequence[30:35])

GTGCA


In [9]:
compute_RBS_affinity(RBS_CANONICAL, sequence, start_codon_position=42, UTR5p_position=8, spacer_range=[5, 11],
                     include_internal_motif_around_start_codon=False, verbose=2)

###
motif AGGAGG region1 25 region2 36
spacer_range [5, 11]
CCACTCGA|AGACGGGACCGCCGGCG|AGCAGGTGCACA|AAGCC|ATG
motif_seq CCTCCT
subseq_list ['AGCAGG', 'GCAGGT', 'CAGGTG', 'AGGTGC', 'GGTGCA', 'GTGCAC', 'TGCACA']
energy_fold_list [['(((.(()).))).', -1.7000000476837158], ['(((.....)))..', -2.0], ['(((....)))...', -2.0], ['.(.((())).)..', -1.600000023841858], ['.(.((.)).)...', -1.2000000476837158], ['.............', 0.0], ['.............', 0.0]]


(-2.0, 26, '(((.....)))..')

In [16]:
import tools
reload(tools)
from tools import RBS_scoring

rs = RBS_scoring(sequence)
rs

\RBS_scoring: finished.




[{'start': 13, 'end': 19, 'raw_score': 3.0, 'norm_score': 3.0},
 {'start': 14, 'end': 20, 'raw_score': 3.0, 'norm_score': 3.0},
 {'start': 15, 'end': 21, 'raw_score': 3.0, 'norm_score': 3.0},
 {'start': 16, 'end': 22, 'raw_score': 3.0, 'norm_score': 3.0},
 {'start': 17, 'end': 23, 'raw_score': 3.0, 'norm_score': 3.0},
 {'start': 18, 'end': 24, 'raw_score': 3.0, 'norm_score': 3.0},
 {'start': 26, 'end': 32, 'raw_score': 2.0, 'norm_score': 2.0},
 {'start': 27, 'end': 33, 'raw_score': 2.0, 'norm_score': 2.0},
 {'start': 28, 'end': 34, 'raw_score': 2.0, 'norm_score': 2.0},
 {'start': 29, 'end': 35, 'raw_score': 2.0, 'norm_score': 2.0},
 {'start': 30, 'end': 36, 'raw_score': 2.0, 'norm_score': 2.0},
 {'start': 31, 'end': 37, 'raw_score': 2.0, 'norm_score': 2.0}]

## visualize example with SeqHighway

In [14]:
from importlib import reload

from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
from Bio.SeqFeature import SeqFeature, FeatureLocation

import seqhighway
reload(seqhighway)
from seqhighway import TrackHighway

In [19]:
# seq = Seq(sequence)
seqrecord = SeqRecord(seq=Seq(sequence), id='example1')

seqfeature = SeqFeature(FeatureLocation(42, 129), type='CDS', strand=+1, qualifiers={'locus_tag':'CDS'})
seqrecord.features.append(seqfeature)

seqfeature = SeqFeature(FeatureLocation(42, 45), type='start_codon', strand=+1)
seqrecord.features.append(seqfeature)

for i in range(len(sequence)):
    elementList = [e1 for e1 in rs if e1['start'] == i]
    if len(elementList) > 0:
        element = elementList[0]
        if element['raw_score'] > 0:
            seqfeature = SeqFeature(FeatureLocation(i, i + 1), type='RBS', strand=+1)
            seqrecord.features.append(seqfeature)

# Create the SeqHighway
seqhighway = TrackHighway(seqrecord, seqrecord_start=0, xmin=0, xmax=145,
                          show_translation=False, codon_table="Bacterial")
seqhighway.add_coordinates_track(ticks=True)

mode = 'inline-block'
# mode = 'fixed_table'
html = seqhighway.to_html(mode=mode, table_width=75)

TrackGroup:read_seqrecord, parsing feature 0 / 14
TrackGroup:read_seqrecord, parsing feature 1 / 14
TrackGroup:read_seqrecord, parsing feature 2 / 14
TrackGroup:read_seqrecord, parsing feature 3 / 14
TrackGroup:read_seqrecord, parsing feature 4 / 14
TrackGroup:read_seqrecord, parsing feature 5 / 14
TrackGroup:read_seqrecord, parsing feature 6 / 14
TrackGroup:read_seqrecord, parsing feature 7 / 14
TrackGroup:read_seqrecord, parsing feature 8 / 14
TrackGroup:read_seqrecord, parsing feature 9 / 14
TrackGroup:read_seqrecord, parsing feature 10 / 14
TrackGroup:read_seqrecord, parsing feature 11 / 14
TrackGroup:read_seqrecord, parsing feature 12 / 14
TrackGroup:read_seqrecord, parsing feature 13 / 14
TrackGroup:organize_tracks
organize_tracks: self.tracks[i].fixed 0 True
features_overlay <seqhighway.seqhighway.Track object at 0x7f13d379a278> <seqhighway.seqhighway.FeatureOverlay object at 0x7f13d3792470>
features_overlay <seqhighway.seqhighway.Track object at 0x7f13d379a278> <seqhighway.seqh

TrackHighway:update_array level 0 track <seqhighway.seqhighway.Track object at 0x7f13d379a278> i 1 j 116
track.seq[j0] C
TrackHighway:update_array level 0 track <seqhighway.seqhighway.Track object at 0x7f13d379a278> i 1 j 117
track.seq[j0] G
TrackHighway:update_array level 0 track <seqhighway.seqhighway.Track object at 0x7f13d379a278> i 1 j 118
track.seq[j0] A
TrackHighway:update_array level 0 track <seqhighway.seqhighway.Track object at 0x7f13d379a278> i 1 j 119
track.seq[j0] G
TrackHighway:update_array level 0 track <seqhighway.seqhighway.Track object at 0x7f13d379a278> i 1 j 120
track.seq[j0] T
TrackHighway:update_array level 0 track <seqhighway.seqhighway.Track object at 0x7f13d379a278> i 1 j 121
track.seq[j0] A
TrackHighway:update_array level 0 track <seqhighway.seqhighway.Track object at 0x7f13d379a278> i 1 j 122
track.seq[j0] C
TrackHighway:update_array level 0 track <seqhighway.seqhighway.Track object at 0x7f13d379a278> i 1 j 123
track.seq[j0] T
TrackHighway:update_array level 

HTMLWriter:seqhighway_html i, j, features 0 17 []
HTMLWriter:seqhighway_html i, j, features 1 17 [None]
HTMLWriter:seqhighway_html i, j, features 2 17 []
HTMLWriter:seqhighway_html i, j, features 3 17 []
HTMLWriter:seqhighway_html i, j, features 0 18 []
HTMLWriter:seqhighway_html i, j, features 1 18 [None]
HTMLWriter:seqhighway_html i, j, features 2 18 []
HTMLWriter:seqhighway_html i, j, features 3 18 []
HTMLWriter:seqhighway_html i, j, features 0 19 []
HTMLWriter:seqhighway_html i, j, features 1 19 []
HTMLWriter:seqhighway_html i, j, features 2 19 []
HTMLWriter:seqhighway_html i, j, features 3 19 []
HTMLWriter:seqhighway_html i, j, features 0 20 []
HTMLWriter:seqhighway_html i, j, features 1 20 []
HTMLWriter:seqhighway_html i, j, features 2 20 ['tick']
HTMLWriter:seqhighway_html i, j, features 3 20 ['tick']
HTMLWriter:seqhighway_html i, j, features 0 21 []
HTMLWriter:seqhighway_html i, j, features 1 21 []
HTMLWriter:seqhighway_html i, j, features 2 21 []
HTMLWriter:seqhighway_html i, 

HTMLWriter:seqhighway_html i, j, features 1 128 []
HTMLWriter:seqhighway_html i, j, features 2 128 []
HTMLWriter:seqhighway_html i, j, features 3 128 []
HTMLWriter:seqhighway_html i, j, features 0 129 []
HTMLWriter:seqhighway_html i, j, features 1 129 []
HTMLWriter:seqhighway_html i, j, features 2 129 []
HTMLWriter:seqhighway_html i, j, features 3 129 []
HTMLWriter:seqhighway_html i, j, features 0 130 []
HTMLWriter:seqhighway_html i, j, features 1 130 []
HTMLWriter:seqhighway_html i, j, features 2 130 ['tick']
HTMLWriter:seqhighway_html i, j, features 3 130 ['tick']
HTMLWriter:seqhighway_html i, j, features 0 131 []
HTMLWriter:seqhighway_html i, j, features 1 131 []
HTMLWriter:seqhighway_html i, j, features 2 131 []
HTMLWriter:seqhighway_html i, j, features 3 131 []
HTMLWriter:seqhighway_html i, j, features 0 132 []
HTMLWriter:seqhighway_html i, j, features 1 132 []
HTMLWriter:seqhighway_html i, j, features 2 132 []
HTMLWriter:seqhighway_html i, j, features 3 132 []
HTMLWriter:seqhighw

In [20]:
from IPython.core.display import HTML, display
display(HTML(html))